In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests
import logging

logic: 
1. first get the max number of external slots: 200
2. then filter all the validators who are eligible to be selected next epoch; if the total number of validators is lower than 200, then every filtered validator should be elected next epoch; if total number is higher than 200, we need to first sort these validators based on their total delegations, and get the top 200 validators.
3. wait for one epoch, to check whether those filtered validators are elected

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)


In [3]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [5]:
def getAllValidator():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result'] 

In [6]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [7]:
def getEpoch():
    method = "hmy_getEpoch"
    params = []
    epoch = get_information(method, params)['result']
    return int(epoch, 16)

In [8]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [9]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [10]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [11]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result']

In [12]:
def getEligibleValidator():
    eligible = []
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
            address = i['validator']['address']
            eligible.append(address)
    return eligible

In [13]:
def E1_test():
    global curr_test
    logging.info(f"Test-E1: A staked validator whose stake is in the top #slots stakes is always considered for election")
    try:
        committees = getCommittees()
        slot = committees['external-slot-count']
        block, last_block = getCurrentAndLastBlock()
        if block == last_block:
            logger.info(f"current block is the last block in epoch, waiting for the new epoch...")
            new_block = block+1
            while block < new_block:
                block = getBlockNumber()
            block, last_block = getCurrentAndLastBlock()
        logger.info(f"current block, {block}, begin collecting eligible validators...")
        # get top #slots nodes who are eligible to elected next epoch
        validator_infos = getAllValidatorInformation()
        eligible = []
        stake = dict()
        for i in validator_infos:
            if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
                address = i['validator']['address']
                eligible.append(address)
                stake[address] = i['total-delegation']

        if len(eligible) > slot:
            sorted_stake = sorted(stake.items(), key=lambda kv: kv[1], reverse = True)
            eligible = [kv[0] for kv in sorted_stake[:slot]]

        # wait for epoch changes
        while block < last_block+1:
            block = getBlockNumber()
        logger.info(f"first block in new epoch reached, {block}, will wait for 5 seconds to begin testing...")
        time.sleep(5)
        # check whether the eligible validators are selected
        validator_infos = getAllValidatorInformation()
        flag = True
        for i in validator_infos:
            if i in eligible:  
                if i['epos-status'] != 'currently elected':
                    logger.warning(f"Test E1: Fail")
                    logger.warning(f"validator {i['validator']['address']} who is eligible to be elected is not elected")
                    flag = False
    except TypeError as e:
        logger.error(f"error: {e}")
    if flag:
        logger.info(f"Test E1: Succeed")
        return True
    else:
        return False

In [ ]:
E1_test()

INFO:root:Test-E1: A staked validator whose stake is in the top #slots stakes is always considered for election
INFO:economic-test:current block, 3462568, begin collecting eligible validators...


current and last block numbers 3462568 3473407
